Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Accessing a Bundle Workflow in Python

In this guide, we'll explore how to access a bundle in Python and use it in your own application. We'll cover the following topics:

1. **Downloading the Bundle**: First, you'll need to download the bundle from its source. This can be done using the `download` API.

2. **Creating a `BundleWorkflow`**: Once you have the bundle, you can create a `BundleWorkflow` object by passing the path to the bundle file as an argument to `create_worflow`.

3. **Getting Properties from the Bundle**: You can then retrieve the properties of the bundle by directly accessing them. For example, to get the version of the bundle, you can use `workflow.version`.

4. **Updating Properties**: If you need to update any of the properties, you can do so by directly overwriting them. For example, to update the max epochs of the bundle, you can use `workflow.max_epochs = 10`.

5. **Using Components in Your Own Pipeline**: Finally, you can use the components from the bundle in your own pipeline by accessing them through the `BundleWorkflow` object.

6. **Utilizing Pretrained Weights from the Bundle**: You can conveniently employ pretrained weights from the bundle and customize them using the `load` API.

7. **A Simple Comparison of the Usage between `ConfigParser` and `BundleWorkflow`**

The bundle documentation and specification can be found here: https://docs.monai.io/en/stable/bundle_intro.html

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"

## Setup imports

In [ ]:
import os
import shutil
import tempfile
from pathlib import Path
import monai
from monai.engines import EnsembleEvaluator
from monai.networks.nets import SegResNet
from monai.transforms import MeanEnsembled, Compose
from monai.config import print_config
from monai.apps import download_and_extract
from monai.bundle import download, create_workflow, ConfigParser, load

print_config()

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/Data


## Download dataset

Downloads and extracts the dataset.  
The dataset comes from http://medicaldecathlon.com/.

In [3]:
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar"
md5 = "410d4a301da4e5b2f6f86ec3ddba524e"

compressed_file = os.path.join(root_dir, "Task09_Spleen.tar")
data_dir = os.path.join(root_dir, "Task09_Spleen")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

## Downloading the Bundle

In [7]:
download(name="spleen_ct_segmentation", bundle_dir=root_dir)

name spleen_ct_segmentation
version None
bundle_dir /workspace/Data
source github
repo None
url None
remove_prefix monai_
progress True
2023-09-06 08:44:11,165 - INFO - --- input summary of monai.bundle.scripts.download ---
2023-09-06 08:44:11,167 - INFO - > name: 'spleen_ct_segmentation'
2023-09-06 08:44:11,167 - INFO - > bundle_dir: '/workspace/Data'
2023-09-06 08:44:11,167 - INFO - > source: 'github'
2023-09-06 08:44:11,167 - INFO - > remove_prefix: 'monai_'
2023-09-06 08:44:11,168 - INFO - > progress: True
2023-09-06 08:44:11,168 - INFO - ---


2023-09-06 08:44:12,165 - INFO - Expected md5 is None, skip md5 check for file /workspace/Data/spleen_ct_segmentation_v0.5.3.zip.
2023-09-06 08:44:12,165 - INFO - File exists: /workspace/Data/spleen_ct_segmentation_v0.5.3.zip, skipped downloading.
2023-09-06 08:44:12,166 - INFO - Writing into directory: /workspace/Data.


## Creating a `BundleWorkflow`
In this section, we demonstrate how to create and initialize a `BundleWorkflow` using the `create_workflow` function. This function supports the creation of both config-based bundles by providing the necessary config files and python-based bundles by specifying a bundle workflow name. The specified name should be a subclass of `BundleWorkflow` and be accessible for import.


In [5]:
config_file = Path(root_dir) / "spleen_ct_segmentation" / "configs" / "train.json"

train_workflow = create_workflow(config_file=str(config_file), workflow_type="train")

workflow_name None
config_file /workspace/Data/spleen_ct_segmentation/configs/train.json
workflow_type train
2023-09-06 09:18:47,393 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-06 09:18:47,395 - INFO - > config_file: '/workspace/Data/spleen_ct_segmentation/configs/train.json'
2023-09-06 09:18:47,396 - INFO - > workflow_type: 'train'
2023-09-06 09:18:47,397 - INFO - ---


2023-09-06 09:18:47,397 - INFO - Setting logging properties based on config: /workspace/Data/spleen_ct_segmentation/configs/logging.conf.


## Getting Properties from the Bundle
To access properties from the bundle, please refer to the list of supported properties available [here](https://docs.monai.io/en/latest/mb_properties.html).

You can also utilize the `add_property` method of the `BundleWorkflow` object to add properties for application requirements checking and access.


In [18]:
# for existing properties
# Note that the properties got from `train_workflow` is already instantiated.
train_preprocessing = train_workflow.train_preprocessing

# for meta information
version = train_workflow.version

# add properties
train_workflow.add_property(name="lr_scheduler", required=True, config_id="lr_scheduler")
print(train_workflow.lr_scheduler)

## Updating Properties
There are two primary methods for updating properties:

1. You can override them during the workflow creation process.
2. Alternatively, you can directly overwrite them after the workflow has been created.

In [16]:
# 1 override them when you create the workflow
bundle_root = root_dir
override = {"epochs": 1, "dataset_dir": str(data_dir), "bundle_root": bundle_root}
train_workflow = create_workflow(config_file=config_file, workflow_type="train", **override)
print("max epochs:", train_workflow.max_epochs)

workflow_name None
config_file /workspace/Data/spleen_ct_segmentation/configs/train.json
workflow_type train
epochs 1
dataset_dir /workspace/Data/Task09_Spleen
bundle_root /workspace/Data
2023-09-06 06:51:08,123 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-06 06:51:08,124 - INFO - > config_file: PosixPath('/workspace/Data/spleen_ct_segmentation/configs/train.json')
2023-09-06 06:51:08,125 - INFO - > workflow_type: 'train'
2023-09-06 06:51:08,126 - INFO - > epochs: 1
2023-09-06 06:51:08,126 - INFO - > dataset_dir: '/workspace/Data/Task09_Spleen'
2023-09-06 06:51:08,126 - INFO - > bundle_root: '/workspace/Data'
2023-09-06 06:51:08,127 - INFO - ---


2023-09-06 06:51:08,127 - INFO - Setting logging properties based on config: /workspace/Data/spleen_ct_segmentation/configs/logging.conf.
max epochs: 1


In [17]:
# 2 directly overwriting them after creating the workflow
train_workflow.max_epochs = 3
train_workflow.bundle_root = bundle_root

# Note that must initialize again after changing the content
train_workflow.initialize()
print("max epochs:", train_workflow.max_epochs)
print("bundle root:", train_workflow.bundle_root)

max epochs: 3
bundle root: /workspace/Data


## Using Components in Your Own Pipeline
If you wish to incorporate additional processing into the bundle's existing post-processing and use it within your custom pipeline, you can follow these steps. A comprehensive example can be found [here](https://github.com/Project-MONAI/tutorials/tree/main/model_zoo/app_integrate_bundle).

In [20]:
n_splits = 3
ensemble_transform = MeanEnsembled(keys=["pred"] * n_splits, output_key="pred")
update_postprocessing = Compose((ensemble_transform, train_workflow.val_postprocessing))
print(update_postprocessing.transforms)

device = train_workflow.device
train_workflow.add_property(name="dataloader", required=True, config_id="train#dataloader")
evaluator = EnsembleEvaluator(
    device=device,
    val_data_loader=train_workflow.dataloader,
    pred_keys=["pred"] * n_splits,
    networks=[train_workflow.network_def.to(train_workflow.device)] * n_splits,
    inferer=train_workflow.train_inferer,
    postprocessing=update_postprocessing,
)

evaluator.run()

(<monai.transforms.post.dictionary.MeanEnsembled object at 0x7ff543020760>, <monai.transforms.compose.Compose object at 0x7ff5430223a0>)
2023-09-06 06:53:28,284 - ignite.engine.engine.EnsembleEvaluator - INFO - Engine run resuming from iteration 0, epoch 0 until 1 epochs


property 'dataloader' already exists in the properties list, overriding it.


2023-09-06 06:53:31,896 - ignite.engine.engine.EnsembleEvaluator - INFO - Epoch[1] Complete. Time taken: 00:00:03.305
2023-09-06 06:53:31,897 - ignite.engine.engine.EnsembleEvaluator - INFO - Engine run complete. Time taken: 00:00:03.612


## Utilizing Pretrained Weights from the Bundle

This function primarily serves to provide an instantiated network by loading pretrained weights from the bundle. You have the flexibility to directly update the parameters or filter the weights. Additionally, it's possible to use your own model instead of the one included in the bundle.


In [ ]:
# directly get an instantiated network that loaded the weights.
model = load(name="brats_mri_segmentation", bundle_dir=root_dir, source="monaihosting")

# directly update the parameters for the model from the bundle.
model = load(name="brats_mri_segmentation", bundle_dir=root_dir, source="monaihosting", in_channels=3, out_channels=1)

# using `exclude_vars` to filter loading weights.
model = load(
    name="brats_mri_segmentation",
    bundle_dir=root_dir,
    source="monaihosting",
    copy_model_args={"exclude_vars": "convInit|conv_final"},
)

# pass model and return an instantiated network that loaded the weights.
my_model = SegResNet(blocks_down=[1, 2, 2, 4], blocks_up=[1, 1, 1], init_filters=16, in_channels=1, out_channels=3)
model = load(name="brats_mri_segmentation", bundle_dir=root_dir, source="monaihosting", model=my_model)

## A Simple Comparison of the Usage between `ConfigParser` and `BundleWorkflow`

### Loading Configuration

In the past, we needed to instantiate a `ConfigParser` and then read the configuration file and meta information using `read_config` and `read_meta` functions. However, now you can skip using `ConfigParser` and directly run `create_workflow` to create a `BundleWorkflow`. This new approach supports both configuration-based and Python-based bundles.


In [8]:
# Using ConfigParser
meta_file = Path(root_dir) / "spleen_ct_segmentation" / "configs" / "metadata.json"
bundle_config = ConfigParser()
bundle_config.read_config(config_file)
bundle_config.read_meta(meta_file)

# Using BundleWorkflow
# config-based
workflow = create_workflow(config_file=str(config_file), workflow_type="train")
# python-based
# more details refer to https://github.com/Project-MONAI/tutorials/tree/main/bundle/python_bundle_workflow
# workflow = create_workflow(workflow_name=scripts.train.TrainWorkflow)

workflow_name None
config_file /workspace/Data/spleen_ct_segmentation/configs/train.json
workflow_type train
2023-09-06 09:19:00,935 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-06 09:19:00,936 - INFO - > config_file: '/workspace/Data/spleen_ct_segmentation/configs/train.json'
2023-09-06 09:19:00,938 - INFO - > workflow_type: 'train'
2023-09-06 09:19:00,939 - INFO - ---


2023-09-06 09:19:00,940 - INFO - Setting logging properties based on config: /workspace/Data/spleen_ct_segmentation/configs/logging.conf.


### Getting and Updating Configuration

Previously, we utilized the `update` method to override configuration content. Now, with `BundleWorkflow`, you can override contents during workflow creation. To obtain an instantiated component, we used to use `get_parsed_content` before. However, now you can access it directly. Additionally, it's worth noting that you can also override the instantiated component, but be sure to initialize it again as needed.


In [ ]:
overrides = {"network_def#in_channels": 1, "lr_scheduler#step_size": 4000, "dataset_dir": str(data_dir), "epochs": 1}

# Using ConfigParser
# override configuration content
bundle_config.config.update(overrides)
# get instantiate the network component
net = bundle_config.get_parsed_content("network_def", instantiate=True)

# Using BundleWorkflow
workflow = create_workflow(config_file=str(config_file), workflow_type="train", **overrides)
# get instantiate the network component
net = workflow.network_def
workflow.network_def = SegResNet(
    blocks_down=[1, 2, 2, 4], blocks_up=[1, 1, 1], init_filters=16, in_channels=1, out_channels=2
)
workflow.initialize()  # re-initialize the workflow after changing the content
print(workflow.network_def)

### Running the Updated Bundle

In the past, running an updated configuration required using `export_config_file` to export the new configuration and using the `run` command. But now, you can streamline the process by directly using the `run` command to execute the new workflow.

In [ ]:
# Using ConfigParser
new_config_path = Path(root_dir) / "spleen_ct_segmentation" / "configs" / "new_train_config.json"
ConfigParser.export_config_file(bundle_config.config, str(new_config_path), indent=2)
monai.bundle.run(run_id="run", init_id=None, final_id=None, meta_file=str(meta_file), config_file=str(new_config_path))

# Using BundleWorkflow
workflow.run()

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)